In [2]:
# Import necessary libraries and modules
import os
import numpy as np
import torch
import pickle
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [11]:
import sys
sys.path.append("/Users/kenzaamara/GithubProjects/syntax-shap/syntaxshap")

import models

In [ ]:

# Import custom modules and functions
from metrics import get_scores, save_scores
import explainers
from explainers.other import LimeTextGeneration, Random
import models
from datasets import generics_kb, inconsistent_negation, rocstories
from utils import arg_parse, fix_random_seed
from utils._exceptions import InvalidAlgorithmError
from utils._filter_data import filter_data
from utils.transformers import parse_prefix_suffix_for_tokenizer

In [12]:

# Define minimum required version of transformers library
MIN_TRANSFORMERS_VERSION = "4.25.1"

# Check if the transformers library meets the minimum version requirement
assert transformers.__version__ >= MIN_TRANSFORMERS_VERSION, f"Please upgrade transformers to version {MIN_TRANSFORMERS_VERSION} or higher."


In [13]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load GPT-2 model
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.to(device)

# Set text generation parameters
model.config.task_specific_params["text-generation"] = {
    "do_sample": True,
    "max_new_tokens": 1,
    "temperature": 0.7,
    "top_k": 50,
    #"no_repeat_ngram_size": 2,
}
model.config.is_decoder = True

In [68]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2", return_tensors="pt")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Initialize TextGeneration model
lmmodel = models.TextGeneration(model, tokenizer, device=device)
#parsed_tokenizer_dict = parse_prefix_suffix_for_tokenizer(lmmodel.tokenizer)
#keep_prefix = parsed_tokenizer_dict['keep_prefix'] # check that keep_prefix is not None, value 0 or 1
#keep_suffix = parsed_tokenizer_dict['keep_suffix'] # check that keep_prefix is not None, value 0 or 1


In [69]:
from captum.attr import LLMAttribution, TextTokenInput
from captum.attr._core.shapley_value import ShapleyValues, ShapleyValueSampling

In [70]:
sv = ShapleyValueSampling(model) 

In [71]:
prompt = "A father is not a"

In [39]:
inp = tokenizer(prompt)
inp

{'input_ids': [32, 2988, 318, 407, 257], 'attention_mask': [1, 1, 1, 1, 1]}

In [54]:
inp = tokenizer("mother")
inp

{'input_ids': [13552], 'attention_mask': [1]}

In [72]:
inp = TextTokenInput(
    prompt, 
    tokenizer,
    skip_tokens=[1],  # skip the special token for the start of the text <s>
)


In [50]:
#inp.to_tensor()
inp.to_model_input()

tensor([[  32, 2988,  318,  407,  257]])

In [75]:

sv_llm_attr = LLMAttribution(sv, tokenizer)

attr_res = sv_llm_attr.attribute(inp, num_trials=1)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [76]:
attr_res.plot_token_attr(show=True)

NameError: name 'attr_res' is not defined